In [1]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import math
import lightgbm as lgb
from collections import Counter  
import time
from scipy.stats import kurtosis,iqr
import seaborn as sns
from scipy import ptp
from tqdm import tqdm
from datetime import timedelta
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
import os
from sklearn.utils import shuffle
from sklearn.externals import joblib
from mpl_toolkits.mplot3d import Axes3D
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.filterwarnings('ignore')

## 专门用来处理网格强度数据

In [24]:
### 北京市重点区域信息
### 添加网格中心坐标之后的数据
area_passenger_info = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/area_passenger_index_processed.csv', index_col=0)
area_passenger_info = area_passenger_info.reset_index(drop=True)
area_passenger_info.iloc[0:10]

,id,index,hour,Center_x,Center_y,Grid_x,Grid_y,area,label,month,day,Grid_x_1,Grid_y_1,index_1,distance
0,3,2.9,0,116.404667,39.935999,116.404148,39.936165,17024.636719,4.1,1,25,116.404148,39.936165,7.9,8.726021e-14
1,3,2.1,1,116.404667,39.935999,116.404148,39.936165,17024.636719,2.3,1,25,116.404148,39.936165,4.9,8.726021e-14
2,3,1.2,2,116.404667,39.935999,116.404148,39.936165,17024.636719,1.6,1,25,116.404148,39.936165,4.9,8.726021e-14
3,3,1.0,3,116.404667,39.935999,116.404148,39.936165,17024.636719,1.6,1,25,116.404148,39.936165,3.2,8.726021e-14
4,3,0.8,4,116.404667,39.935999,116.404148,39.936165,17024.636719,1.7,1,25,116.404148,39.936165,3.8,8.726021e-14
5,3,1.3,5,116.404667,39.935999,116.404148,39.936165,17024.636719,2.0,1,25,116.404148,39.936165,2.6,8.726021e-14
6,3,1.6,6,116.404667,39.935999,116.404148,39.936165,17024.636719,3.4,1,25,116.404148,39.936165,4.2,8.726021e-14
7,3,2.1,7,116.404667,39.935999,116.404148,39.936165,17024.636719,5.7,1,25,116.404148,39.936165,5.8,8.726021e-14
8,3,4.3,8,116.404667,39.935999,116.404148,39.936165,17024.636719,7.0,1,25,116.404148,39.936165,7.5,8.726021e-14
9,3,5.7,9,116.404667,39.935999,116.404148,39.936165,17024.636719,9.6,1,25,116.404148,39.936165,10.6,8.726021e-14


In [25]:
Counter(area_passenger_info['id'])

Counter({3: 720})

In [22]:
cols = ['id', 'date/hour', 'index']
area_passenger_index = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/area_passenger_index.csv', names=cols)
area_passenger_index.iloc[0:10]

,id,date/hour,index
0,1,2020011700,1.8
1,1,2020011701,1.5
2,1,2020011702,1.3
3,1,2020011703,1.3
4,1,2020011704,1.7
5,1,2020011705,1.2
6,1,2020011706,2.9
7,1,2020011707,4.7
8,1,2020011708,6.1
9,1,2020011709,5.2


In [23]:
area_passenger_index.shape

(717840, 3)

In [26]:
### 将网格联系强度添加至表格中
### 在每个时刻，计算某一区域与其他所有区域的网格联系强度之和（in），然后减去该区域去往其他区域的网格联系强度之和（out）
cols = ['hour', 'start_grid_x', 'start_grid_y', 'end_grid_x', 'end_grid_y', 'index']
grid_strength = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/datafountain_competition_od.txt', names=cols, encoding='gb2312', sep='\t')
grid_strength.shape

(9341613, 6)

In [6]:
### 根据每个网格中心点与重点区域的距离，给每条网格联系强度打上id，以便后续处理
def add_id_to_grid(df, area_passenger_info):
    start_id = np.zeros((df.shape[0], 1))-999
    start_id = pd.DataFrame(start_id)
    start_id.columns = ['start_id']
    
    end_id = np.zeros((df.shape[0], 1))-999
    end_id = pd.DataFrame(end_id)
    end_id.columns = ['end_id']
    
    area_passenger_info['dis'] = 0
    
    for index, row in tqdm(df.iterrows()):
        if (start_id.iloc[index]['start_id']==-999):
            ### 遍历找出最近的出发地区
            ### 批量计算距离
            area_passenger_info['dis'] = (row['start_grid_x']-area_passenger_info['Grid_x'])**2 + (row['start_grid_y']-area_passenger_info['Grid_y'])**2
            ### 找出最小距离
            min_dis = area_passenger_info['dis'].min()
            ### 将最小距离对应的id赋给start_id
            start_id.iloc[index]['start_id'] = area_passenger_info[area_passenger_info['dis']==min_dis]['id'].values[0]
            ### 以出发地坐标为依据，给所有同地段的数据打上相同id
            start_id[(df['start_grid_x']==row['start_grid_x']) & (df['start_grid_y']==row['start_grid_y'])]['start_id'] = start_id.iloc[index]['start_id']
    
    for index, row in tqdm(df.iterrows()):
        if (end_id.iloc[index]['end_id']==-999):
            ### 遍历找出最近的出发地区
            ### 批量计算距离
            area_passenger_info['dis'] = (row['end_grid_x']-area_passenger_info['Grid_x'])**2 + (row['end_grid_y']-area_passenger_info['Grid_y'])**2
            ### 找出最小距离
            min_dis = area_passenger_info['dis'].min()
            ### 将最小距离对应的id赋给start_id
            end_id.iloc[index]['end_id'] = area_passenger_info[area_passenger_info['dis']==min_dis]['id'].values[0]
            ### 以出发地坐标为依据，给所有同地段的数据打上相同id
            end_id[(df['end_grid_x']==row['end_grid_x']) & (df['end_grid_y']==row['end_grid_y'])]['end_id'] = end_id.iloc[index]['end_id']
            
            
            
    ### 合并
    df = pd.concat([df, start_id], axis=1)
    df = pd.concat([df, end_id], axis=1)

    return df

In [7]:
grid_strength = add_id_to_grid(df=grid_strength, area_passenger_info=area_passenger_info)

20it [00:00, 21.12it/s]
20it [00:00, 21.63it/s]


In [8]:
### 保存到本地
grid_strength.to_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/grid_strength_processed.csv', index=None)